In [ ]:
# Importar MySQL Connector/Python 
import mysql.connector as connector
from mysql.connector import Error, pooling

# Tarea 1: Establecer una agrupación de conexiones

# Configuración de la base de datos
dbconfig = {
    "database": "little_lemon_db",
    "user": "your_username",
    "password": "your_password"
}

# Crear la agrupación de conexiones
try:
    pool = pooling.MySQLConnectionPool(pool_name="pool_b", pool_size=2, **dbconfig)
    print("Connection pool created successfully.")
except Error as e:
    print(f"Error creating connection pool: {e}")

# Obtener una conexión del pool
connection = pool.get_connection()
cursor = connection.cursor()

# Crear base de datos y tablas
cursor.execute("CREATE DATABASE IF NOT EXISTS little_lemon_db")
cursor.execute("USE little_lemon_db")

# Definir las consultas para crear tablas
create_menuitem_table = """CREATE TABLE IF NOT EXISTS MenuItems (
ItemID INT AUTO_INCREMENT PRIMARY KEY,
Name VARCHAR(200),
Type VARCHAR(100),
Price INT
);"""

create_menu_table = """CREATE TABLE IF NOT EXISTS Menus (
MenuID INT,
ItemID INT,
Cuisine VARCHAR(100),
PRIMARY KEY (MenuID, ItemID)
);"""

create_booking_table = """CREATE TABLE IF NOT EXISTS Bookings (
BookingID INT AUTO_INCREMENT PRIMARY KEY,
TableNo INT,
GuestFirstName VARCHAR(100) NOT NULL,
GuestLastName VARCHAR(100) NOT NULL,
BookingSlot TIME NOT NULL,
EmployeeID INT
);"""

create_orders_table = """CREATE TABLE IF NOT EXISTS Orders (
OrderID INT PRIMARY KEY,
TableNo INT,
MenuID INT,
BookingID INT,
BillAmount INT,
Quantity INT
);"""

create_employees_table = """CREATE TABLE IF NOT EXISTS Employees (
EmployeeID INT AUTO_INCREMENT PRIMARY KEY,
Name VARCHAR(255),
Role VARCHAR(255),
Address VARCHAR(255),
Contact_Number INT,
Email VARCHAR(255),
Annual_Salary VARCHAR(255)
);"""

# Crear tablas
cursor.execute(create_menuitem_table)
cursor.execute(create_menu_table)
cursor.execute(create_booking_table)
cursor.execute(create_orders_table)
cursor.execute(create_employees_table)

# Insertar datos en las tablas
insert_menuitems = """
INSERT INTO MenuItems (ItemID, Name, Type, Price)
VALUES
(1, 'Olives','Starters',5),
(2, 'Flatbread','Starters', 5),
(3, 'Minestrone', 'Starters', 8),
(4, 'Tomato bread','Starters', 8),
(5, 'Falafel', 'Starters', 7),
(6, 'Hummus', 'Starters', 5),
(7, 'Greek salad', 'Main Courses', 15),
(8, 'Bean soup', 'Main Courses', 12),
(9, 'Pizza', 'Main Courses', 15),
(10, 'Greek yoghurt','Desserts', 7),
(11, 'Ice cream', 'Desserts', 6),
(12, 'Cheesecake', 'Desserts', 4),
(13, 'Athens White wine', 'Drinks', 25),
(14, 'Corfu Red Wine', 'Drinks', 30),
(15, 'Turkish Coffee', 'Drinks', 10),
(16, 'Turkish Coffee', 'Drinks', 10),
(17, 'Kabasa', 'Main Courses', 17);"""

insert_menu = """
INSERT INTO Menus (MenuID, ItemID, Cuisine)
VALUES
(1, 1, 'Greek'),
(1, 7, 'Greek'),
(1, 10, 'Greek'),
(1, 13, 'Greek'),
(2, 3, 'Italian'),
(2, 9, 'Italian'),
(2, 12, 'Italian'),
(2, 15, 'Italian'),
(3, 5, 'Turkish'),
(3, 17, 'Turkish'),
(3, 11, 'Turkish'),
(3, 16, 'Turkish');"""

insert_bookings = """
INSERT INTO Bookings (BookingID, TableNo, GuestFirstName, 
GuestLastName, BookingSlot, EmployeeID)
VALUES
(1, 12, 'Anna','Iversen','19:00:00',1),
(2, 12, 'Joakim', 'Iversen', '19:00:00', 1),
(3, 19, 'Vanessa', 'McCarthy', '15:00:00', 3),
(4, 15, 'Marcos', 'Romero', '17:30:00', 4),
(5, 5, 'Hiroki', 'Yamane', '18:30:00', 2),
(6, 8, 'Diana', 'Pinto', '20:00:00', 5);"""

insert_orders = """
INSERT INTO Orders (OrderID, TableNo, MenuID, BookingID, Quantity, BillAmount)
VALUES
(1, 12, 1, 1, 2, 86),
(2, 19, 2, 2, 1, 37),
(3, 15, 2, 3, 1, 37),
(4, 5, 3, 4, 1, 40),
(5, 8, 1, 5, 1, 43);"""

insert_employees = """
INSERT INTO Employees (EmployeeID, Name, Role, Address, Contact_Number, Email, Annual_Salary)
VALUES
(1,'Mario Gollini','Manager','724, Parsley Lane, Old Town, Chicago, IL',351258074,'Mario.g@littlelemon.com','$70,000'),
(2,'Adrian Gollini','Assistant Manager','334, Dill Square, Lincoln Park, Chicago, IL',351474048,'Adrian.g@littlelemon.com','$65,000'),
(3,'Giorgos Dioudis','Head Chef','879 Sage Street, West Loop, Chicago, IL',351970582,'Giorgos.d@littlelemon.com','$50,000'),
(4,'Fatma Kaya','Assistant Chef','132  Bay Lane, Chicago, IL',351963569,'Fatma.k@littlelemon.com','$45,000'),
(5,'Elena Salvai','Head Waiter','989 Thyme Square, EdgeWater, Chicago, IL',351074198,'Elena.s@littlelemon.com','$40,000'),
(6,'John Millar','Receptionist','245 Dill Square, Lincoln Park, Chicago, IL',351584508,'John.m@littlelemon.com','$35,000');"""

# Insertar datos en las tablas
cursor.execute(insert_menuitems)
cursor.execute(insert_menu)
cursor.execute(insert_bookings)
cursor.execute(insert_orders)
cursor.execute(insert_employees)
connection.commit()

# Devolver la conexión al pool
cursor.close()
connection.close()

# Obtener una nueva conexión del pool para los procedimientos almacenados
connection = pool.get_connection()
cursor = connection.cursor()

# Tarea 2: Implementar el procedimiento almacenado PeakHours

create_peak_hours_procedure = """
CREATE PROCEDURE PeakHours()
BEGIN
    SELECT HOUR(BookingSlot) AS Hour, COUNT(*) AS NumberOfBookings
    FROM Bookings
    GROUP BY HOUR(BookingSlot)
    ORDER BY NumberOfBookings DESC;
END;
"""

cursor.execute(create_peak_hours_procedure)
connection.commit()

# Llamar al procedimiento almacenado PeakHours
cursor.callproc("PeakHours")
results = cursor.stored_results()
for result in results:
    columns = [i[0] for i in result.description]
    print("Column Names:", columns)
    for row in result:
        print(row)

# Devolver la conexión al pool
cursor.close()
connection.close()

# Obtener una nueva conexión del pool para el siguiente procedimiento almacenado
connection = pool.get_connection()
cursor = connection.cursor()

# Tarea 3: Implementar el procedimiento almacenado GuestStatus

create_guest_status_procedure = """
CREATE PROCEDURE GuestStatus()
BEGIN
    SELECT CONCAT(GuestFirstName, ' ', GuestLastName) AS GuestName,
    CASE
        WHEN e.Role IN ('Manager', 'Assistant Manager') THEN 'Ready to pay'
        WHEN e.Role = 'Head Chef' THEN 'Ready to serve'
        WHEN e.Role = 'Assistant Chef' THEN 'Preparing Order'
        WHEN e.Role = 'Head Waiter' THEN 'Order served'
    END AS OrderStatus
    FROM Bookings b
    LEFT JOIN Employees e ON b.EmployeeID = e.EmployeeID;
END;
"""

cursor.execute(create_guest_status_procedure)
connection.commit()

# Llamar al procedimiento almacenado GuestStatus
cursor.callproc("GuestStatus")
results = cursor.stored_results()
for result in results:
    columns = [i[0] for i in result.description]
    print("Column Names:", columns)
    for row in result:
        print(row)

# Devolver la conexión al pool
cursor.close()
connection.close()

# Obtener una nueva conexión del pool para el siguiente procedimiento almacenado
connection = pool.get_connection()
cursor = connection.cursor()

# Tarea 4: Crear el procedimiento almacenado BasicSalesReport

create_basic_sales_report_procedure = """
CREATE PROCEDURE BasicSalesReport()
BEGIN
    SELECT 
        SUM(BillAmount) AS TotalSales,
        AVG(BillAmount) AS AverageSale,
        MIN(BillAmount) AS MinBill,
        MAX(BillAmount) AS MaxBill
    FROM Orders;
END;
"""

cursor.execute(create_basic_sales_report_procedure)
connection.commit()

# Llamar al procedimiento almacenado BasicSalesReport
cursor.callproc("BasicSalesReport")
results = cursor.stored_results()
for result in results:
    columns = [i[0] for i in result.description]
    print("Column Names:", columns)
    for row in result:
        print(row)

# Devolver la conexión al pool
cursor.close()
connection.close()

# Obtener una nueva conexión del pool para mostrar próximas reservas
connection = pool.get_connection()
cursor = connection.cursor(buffered=True)

# Tarea 5: Mostrar las próximas reservas

# Consulta para obtener las próximas reservas
query = """
SELECT BookingSlot, CONCAT(GuestFirstName, ' ', GuestLastName) AS GuestName, 
       CONCAT('Assigned to: ', e.Name, ' [', e.Role, ']') AS EmployeeInfo
FROM Bookings b
LEFT JOIN Employees e ON b.EmployeeID = e.EmployeeID
ORDER BY BookingSlot
LIMIT 3;
"""

cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

# Devolver la conexión al pool
cursor.close()
connection.close()
